In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 3.4MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpn65a5v__
100%|██████████| 231508/231508 [00:00<00:00, 2599305.96B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpn65a5v__ to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpn65a5v__
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f0006

In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F

In [0]:
text ='what is a pug'

In [0]:
zz = tokenizer.tokenize(text)


In [0]:
class BertLayerNorm(nn.Module):
        def __init__(self, hidden_size, eps=1e-12):
            """Construct a layernorm module in the TF style (epsilon inside the square root).
            """
            super(BertLayerNorm, self).__init__()
            self.weight = nn.Parameter(torch.ones(hidden_size))
            self.bias = nn.Parameter(torch.zeros(hidden_size))
            self.variance_epsilon = eps

        def forward(self, x):
            u = x.mean(-1, keepdim=True)
            s = (x - u).pow(2).mean(-1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.variance_epsilon)
            return self.weight * x + self.bias
        

class BertForSequenceClassification(nn.Module):
    
    def __init__(self, num_labels=2):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [0]:
from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

num_labels = 2
model = BertForSequenceClassification(num_labels)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(zz)])

logits = model(tokens_tensor)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmptecryy3b
100%|██████████| 407873900/407873900 [00:05<00:00, 70454656.21B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmptecryy3b to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmptecryy3b
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb7

In [0]:
logits

tensor([[-0.7852,  0.0474]], grad_fn=<AddmmBackward>)

In [0]:
import pandas as pd

dat = pd.read_csv('/content/custom_bert_pipeline/IMDB Dataset.csv')

# New Section

In [0]:
dat.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
zz = tokenizer.tokenize(dat.review[1])
z1z = tokenizer.tokenize(dat.review[3])

[zz,z1z ]

[['a',
  'wonderful',
  'little',
  'production',
  '.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  'the',
  'filming',
  'technique',
  'is',
  'very',
  'una',
  '##ss',
  '##uming',
  '-',
  'very',
  'old',
  '-',
  'time',
  '-',
  'bbc',
  'fashion',
  'and',
  'gives',
  'a',
  'comforting',
  ',',
  'and',
  'sometimes',
  'discomfort',
  '##ing',
  ',',
  'sense',
  'of',
  'realism',
  'to',
  'the',
  'entire',
  'piece',
  '.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  'the',
  'actors',
  'are',
  'extremely',
  'well',
  'chosen',
  '-',
  'michael',
  'sheen',
  'not',
  'only',
  '"',
  'has',
  'got',
  'all',
  'the',
  'polar',
  '##i',
  '"',
  'but',
  'he',
  'has',
  'all',
  'the',
  'voices',
  'down',
  'pat',
  'too',
  '!',
  'you',
  'can',
  'truly',
  'see',
  'the',
  'seam',
  '##less',
  'editing',
  'guided',
  'by',
  'the',
  'references',
  'to',
  'williams',
  "'",
  'diary',
  'entries',
  ',',
  'not',
  'only

In [0]:
zzz = tokenizer.convert_tokens_to_ids(zz)
zzzz = tokenizer.convert_tokens_to_ids(z1z)

In [0]:
zzz

[1037,
 6919,
 2210,
 2537,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 1996,
 7467,
 6028,
 2003,
 2200,
 14477,
 4757,
 24270,
 1011,
 2200,
 2214,
 1011,
 2051,
 1011,
 4035,
 4827,
 1998,
 3957,
 1037,
 16334,
 1010,
 1998,
 2823,
 17964,
 2075,
 1010,
 3168,
 1997,
 15650,
 2000,
 1996,
 2972,
 3538,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 1996,
 5889,
 2024,
 5186,
 2092,
 4217,
 1011,
 2745,
 20682,
 2025,
 2069,
 1000,
 2038,
 2288,
 2035,
 1996,
 11508,
 2072,
 1000,
 2021,
 2002,
 2038,
 2035,
 1996,
 5755,
 2091,
 6986,
 2205,
 999,
 2017,
 2064,
 5621,
 2156,
 1996,
 25180,
 3238,
 9260,
 8546,
 2011,
 1996,
 7604,
 2000,
 3766,
 1005,
 9708,
 10445,
 1010,
 2025,
 2069,
 2003,
 2009,
 2092,
 4276,
 1996,
 3666,
 2021,
 2009,
 2003,
 1037,
 27547,
 2135,
 2517,
 1998,
 2864,
 3538,
 1012,
 1037,
 3040,
 3993,
 2537,
 2055,
 2028,
 1997,
 1996,
 2307,
 3040,
 1005,
 1055,
 1997,
 4038,
 1998,
 2010,
 2166,
 1012,
 1026,
 7987,
 101

In [0]:

tokens_tensor = torch.tensor([zzz,zzz])


In [0]:
tokens_tensor

tensor([[ 1037,  6919,  2210,  2537,  1012,  1026,  7987,  1013,  1028,  1026,
          7987,  1013,  1028,  1996,  7467,  6028,  2003,  2200, 14477,  4757,
         24270,  1011,  2200,  2214,  1011,  2051,  1011,  4035,  4827,  1998,
          3957,  1037, 16334,  1010,  1998,  2823, 17964,  2075,  1010,  3168,
          1997, 15650,  2000,  1996,  2972,  3538,  1012,  1026,  7987,  1013,
          1028,  1026,  7987,  1013,  1028,  1996,  5889,  2024,  5186,  2092,
          4217,  1011,  2745, 20682,  2025,  2069,  1000,  2038,  2288,  2035,
          1996, 11508,  2072,  1000,  2021,  2002,  2038,  2035,  1996,  5755,
          2091,  6986,  2205,   999,  2017,  2064,  5621,  2156,  1996, 25180,
          3238,  9260,  8546,  2011,  1996,  7604,  2000,  3766,  1005,  9708,
         10445,  1010,  2025,  2069,  2003,  2009,  2092,  4276,  1996,  3666,
          2021,  2009,  2003,  1037, 27547,  2135,  2517,  1998,  2864,  3538,
          1012,  1037,  3040,  3993,  2537,  2055,  

In [0]:
logits = model(tokens_tensor)
logits

tensor([[-0.2061, -0.1864],
        [-0.4167, -0.1316]], grad_fn=<AddmmBackward>)

In [0]:
import torch.nn.functional as F

F.softmax(logits,dim=1)

tensor([[0.4951, 0.5049],
        [0.4292, 0.5708]], grad_fn=<SoftmaxBackward>)

In [0]:
from sklearn.model_selection import train_test_split
X = dat['review']
y = dat['sentiment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


In [0]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()
y_test = pd.get_dummies(y_test).values.tolist()


In [0]:

max_seq_length = 256
class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):
        
        self.x_y_list = x_y_list
        self.transform = transform
        
    def __getitem__(self,index):
        
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
            
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)

        padding = [0] * (max_seq_length - len(ids_review))
        
        ids_review += padding
        
        assert len(ids_review) == max_seq_length
        
        #print(ids_review)
        ids_review = torch.tensor(ids_review)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [0]:

batch_size = 16

train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            
            sentiment_corrects = 0
            
            
            # Iterate over data.
            for inputs, sentiment in dataloaders_dict[phase]:
                #inputs = inputs
                #print(len(inputs),type(inputs),inputs)
                #inputs = torch.from_numpy(np.array(inputs)).to(device) 
                inputs = inputs.to(device) 

                sentiment = sentiment.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs)

                    outputs = F.softmax(outputs,dim=1)
                    
                    loss = criterion(outputs, torch.max(sentiment.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

                
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(sentiment, 1)[1])

                
            epoch_loss = running_loss / dataset_sizes[phase]

            
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
model.to(device)

#model.freeze_bert_encoder()
#model.classifier.weight.requires_grad = True
model

#    def freeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = False
#    
#    def unfreeze_bert_encoder(self):
#        for param in self.bert.parameters():
#            param.requires_grad = True

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


In [0]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,  num_epochs=10)

starting
Epoch 0/9
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [0]:
model